# Sea surface height

Plot the SSH

In [13]:
import sys
sys.path.insert(0, '..')
import os
import glob
from filament import filament
import datetime
import netCDF4
import cmocean
import logging
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
reload(filament)
plt.style.use("filament.mplstyle")

In [2]:
logger = logging.getLogger("altimetryPlot")
logger.setLevel(logging.INFO)
logging.info("Starting")
plotvis = 0

In [31]:
regionname= "BloomJune2024"
domain = filament.regions["canarydomain"]
figdir = f"../figures/SST/{regionname}/"
datadir = f"/home/ctroupin/data/SST/{regionname}/"
datafilelist = sorted(glob.glob(os.path.join(datadir, "nrt_europe_allsat_phy_l4*.nc")))
logger.info(f"Found {len(datafilelist)} files")

domain = filament.regions["canarydomain"]
myproj = ccrs.Mercator(central_longitude=0.5 * (domain[0] + domain[1]), min_latitude=domain[2], 
                       max_latitude=domain[3])

INFO:altimetryPlot:Found 6 files


In [32]:
if not(os.path.exists(figdir)):
    os.makedirs(figdir)

In [35]:
for datafile in datafilelist:
    ssh = filament.Altimetry()
    ssh.read_from_aviso(datafile)
    ssh.get_speed()
    figname = ssh.get_figname()

    goodlon = np.where((ssh.lon <= domain[1] + 1) & (ssh.lon >= domain[0] -1.))[0]
    goodlat = np.where((ssh.lat <= domain[3] + 1) & (ssh.lat >= domain[2] -1.))[0]
    ssh_domain = ssh.sla[goodlat][:,goodlon]
    u = ssh.u[goodlat][:,goodlon]
    v = ssh.v[goodlat][:,goodlon]

    fig = plt.figure()
    ax = plt.subplot(111, projection=myproj)
    ax.set_extent(domain)
    pcm = ax.pcolor(ssh.lon[goodlon], ssh.lat[goodlat], ssh_domain - ssh_domain.mean(), 
                    transform=filament.datacrs,
                    vmin=-.1, vmax=.1, cmap=plt.cm.RdBu_r, zorder=2)
    ax.streamplot(ssh.lon[goodlon], ssh.lat[goodlat], u, v, density=5, linewidth=.5, zorder=3, color=".2",
                 transform=filament.datacrs)
    ax.add_feature(filament.coast_f, lw=.25, color=".5", zorder=4)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=.5, color='.25', linestyle='--', zorder=2)
    gl.top_labels = False
    gl.right_labels = False
    
    cb = plt.colorbar(pcm, shrink=.75, extend="both")
    cb.set_label("m", rotation=0, ha="left")
    ax.set_title(f"Sea level anomaly and streamlines (CMEMS)\n {ssh.date[0].strftime('%Y-%m-%d')}")
    plt.savefig(os.path.join(figdir, figname))
    plt.close()

/home/ctroupin/Projects/Filaments/Filament-Submesoscale/notebooks/../filament/filament.py:1120: RuntimeWarning: invalid value encountered in sqrt
  self.speed = np.ma.masked_greater(self.speed, 1.5)
/tmp/ipykernel_737164/3916038956.py:27: MatplotlibDeprecationWarning: Getting the array from a PolyQuadMesh will return the full array in the future (uncompressed). To get this behavior now set the PolyQuadMesh with a 2D array .set_array(data2d).
  cb = plt.colorbar(pcm, shrink=.75, extend="both")
/home/ctroupin/Projects/Filaments/Filament-Submesoscale/notebooks/../filament/filament.py:1120: RuntimeWarning: invalid value encountered in sqrt
  self.speed = np.ma.masked_greater(self.speed, 1.5)
/tmp/ipykernel_737164/3916038956.py:27: MatplotlibDeprecationWarning: Getting the array from a PolyQuadMesh will return the full array in the future (uncompressed). To get this behavior now set the PolyQuadMesh with a 2D array .set_array(data2d).
  cb = plt.colorbar(pcm, shrink=.75, extend="both")
/hom

In [29]:
ssh.date[0].strftime('%Y-%m-%d %H:%M:%S')

'2024-06-28 00:00:00'